In [1]:
import numpy as np
from typing import *
from dataclasses import dataclass
import os
import pandas as pd
import scipy.stats as stats

In [2]:
@dataclass
class run_data:
    last_val_mean: float
    iter_to_vtr_mean: float
    nfev_mean: float
    last_val_std: float
    iter_to_vtr_std: float
    nfev_std: float
    time_mean: float
    time_std: float

# pandas dataframe template columns
columns = ["last val mean", "iter to vtr mean", "nfev mean", "last val std", "iter to vtr std", "nfev std", "time mean", "time std"]
statistics_test_columns = ["last val pvalue", "iter to vtr pvalue", "nfev pvalue", "time pvalue"]
test_names = [
    "sphere", "weighted_sphere", "schwefel_1_2", "schwefel_2_3",
    "rotated_hyper_ellipsoid", "rosenbrock", "pow_sum", "easom",
    "ackley", "rastrigin", "griewangk", "dejong_5",
    "dejong_3"
]

In [3]:
def get_run_data(filepath):
    with open(filepath, "r") as f:
        data = f.read().splitlines()
    last_val = float(data[0])
    iter_to_vtr = int(data[1])
    nfev = int(data[2])
    time = float(data[3])
    return last_val, iter_to_vtr, nfev, time

def read_all_data_for_test(algorithm_name:str, test_name: str, n: int = 30):
    last_val_array = []
    iter_to_vtr_arr = []
    nfev_arr = []
    time_arr = []

    for i in range(n):
        filepath = f"results/{algorithm_name}/{test_name}/{i}.txt"
        last_val, iter_to_vtr, nfev, time = get_run_data(filepath)
        last_val_array.append(last_val)
        iter_to_vtr_arr.append(iter_to_vtr)
        time_arr.append(time)
        nfev_arr.append(nfev)

    last_val_array = np.array(last_val_array, np.float64)
    iter_to_vtr_arr = np.array(iter_to_vtr_arr, np.float64)
    nfev_arr = np.array(nfev_arr, np.float64)
    time_arr = np.array(time_arr, np.float64)

    return last_val_array, iter_to_vtr_arr, nfev_arr, time_arr

def compute_all_for_test(algorithm_name:str, test_name: str, n: int = 30):

    last_val_array, iter_to_vtr_arr, nfev_arr, time_arr = read_all_data_for_test(algorithm_name, test_name, n)

    last_val_mean = np.mean(last_val_array)
    last_val_std = np.std(last_val_array)

    iter_to_vtr_mean = np.mean(iter_to_vtr_arr)
    iter_to_vtr_std = np.std(iter_to_vtr_arr)

    nfev_mean = np.mean(nfev_arr)
    nfev_std = np.std(nfev_arr)

    time_mean = np.mean(time_arr)
    time_std = np.std(time_arr)

    data = run_data(last_val_mean, iter_to_vtr_mean, nfev_mean, last_val_std, iter_to_vtr_std, nfev_std, time_mean, time_std)

    return data



In [4]:
def make_stats_table(algo1_name: str, algo2_name: str, test_names: List[str] = test_names):
    statistics_test_df = pd.DataFrame(columns=statistics_test_columns)
    for test_name in test_names:
        print(f"Test name: {test_name}")
        last_val_less, iter_to_vtr_less, nfev_less, time_less = compare_algorithms(algo1_name, algo2_name, test_name)
        last_val_pvalue = last_val_less.pvalue if last_val_less is not None else None
        iter_to_vtr_pvalue = iter_to_vtr_less.pvalue if iter_to_vtr_less is not None else None
        nfev_pvalue = nfev_less.pvalue if nfev_less is not None else None
        time_pvalue = time_less.pvalue if time_less is not None else None
        statistics_test_df.loc[test_name.replace("_", " ")] = [last_val_pvalue, iter_to_vtr_pvalue, nfev_pvalue, time_pvalue]
    return statistics_test_df

In [10]:
def make_values_table(algo_name: str, test_names: List[str] = test_names):
    values_df = pd.DataFrame(columns=columns)

    for test_name in os.listdir(f"results/{algo_name}"):
        if test_name not in test_names:
            continue

        data = compute_all_for_test(algo_name, test_name)
        values_df.loc[test_name.replace("_", " ")] = [
            data.last_val_mean,
            data.iter_to_vtr_mean,
            data.nfev_mean,
            data.last_val_std,
            data.iter_to_vtr_std,
            data.nfev_std,
            data.time_mean,
            data.time_std
        ]

    return values_df

In [11]:
make_values_table("ttp_de_sa_2")

,last val mean,iter to vtr mean,nfev mean,last val std,iter to vtr std,nfev std,time mean,time std
schwefel 1 2,4.835481e-17,60.933333,14297.633333,1.738446e-16,11.048177,2055.950769,0.770191,0.110274
rotated hyper ellipsoid,6.462255e-58,41.333333,10316.500000,1.919758e-57,4.671426,1873.696538,3.070522,0.560805
ackley,1.501022e-14,237.066667,35099.400000,1.406205e-15,21.436625,3493.921766,1.115078,0.107473
dejong 5,9.980038e-01,15.700000,668.000000,1.516853e-16,2.968164,118.726577,0.015408,0.002640
dejong 3,0.000000e+00,58.800000,2392.000000,0.000000e+00,3.944617,157.784663,0.053581,0.003475
schwefel 2 3,3.818270e-04,191.833333,109502.633333,9.767181e-12,11.684986,7923.498308,1.956024,0.138482
sphere,1.873583e-62,3.300000,525.400000,5.433269e-62,2.396525,229.006201,0.010150,0.004313
weighted sphere,1.578390e-59,48.733333,11915.533333,4.124422e-59,5.876129,1813.343978,0.135625,0.017109
pow sum,1.046378e-76,52.666667,5128.366667,2.666474e-76,6.257440,2162.491503,0.118282,0.039997
rosenbrock,6.338919e-15,318.000000,77172.833333,1.416896e-14,44.183707,12335.362886,1.429947,0.221831


In [7]:
make_values_table("de_da")

,last val mean,iter to vtr mean,nfev mean,last val std,iter to vtr std,nfev std,time mean,time std
schwefel 1 2,1.407093e-14,138.700000,26772.366667,1.727412e-14,18.160672,3533.875366,2.195855,0.290406
rotated hyper ellipsoid,2.758236e-29,141.466667,21153.200000,3.884724e-29,14.662272,2103.791837,1.956137,0.189677
ackley,1.005919e-01,384.800000,50376.866667,3.036150e-01,213.834422,11817.838380,1.600632,0.426450
dejong 5,1.031138e+00,45.533333,1861.333333,1.784333e-01,177.252500,7090.100015,0.043204,0.167801
dejong 3,0.000000e+00,49.266667,2010.666667,0.000000e+00,6.292500,251.699998,0.045428,0.007480
schwefel 2 3,3.948326e+00,215.400000,44491.100000,2.126033e+01,146.758668,16899.047560,0.878110,0.358657
sphere,2.035899e-35,4.900000,572.866667,4.994175e-35,3.927255,185.111090,0.011654,0.004416
weighted sphere,1.227194e-32,144.266667,21541.100000,1.097175e-32,9.715738,1734.671484,0.269840,0.019684
pow sum,2.206166e-47,89.200000,37501.866667,9.445805e-47,10.287209,5284.651668,0.707702,0.096462
rosenbrock,9.498993e-10,671.666667,472662.900000,4.337129e-09,108.599980,72353.780017,8.033742,1.238137


In [8]:
make_values_table("ge")

,last val mean,iter to vtr mean,nfev mean,last val std,iter to vtr std,nfev std,time mean,time std
schwefel 1 2,7.927917e+02,1000.000000,40378.266667,3.756026e+02,0.000000,11.051194,13.678487,0.159284
rotated hyper ellipsoid,2.604423e-17,200.233333,8089.900000,1.344935e-17,8.747127,349.943457,2.689057,0.198244
ackley,3.085055e-11,318.033333,12805.766667,1.072601e-10,20.105527,805.361148,0.724714,0.056247
dejong 5,9.980038e-01,195.033333,7954.866667,1.178346e-08,138.168372,5590.000732,0.325341,0.239657
dejong 3,0.000000e+00,27.066667,1162.666667,0.000000e+00,8.988635,359.545392,0.047273,0.016510
schwefel 2 3,1.069657e+04,1000.000000,40365.866667,3.569357e+01,0.000000,26.248471,1.811865,0.057593
sphere,2.028771e-17,182.333333,7373.666667,8.058770e-18,9.217134,368.829078,0.323011,0.026333
weighted sphere,1.034165e+01,1000.000000,40246.433333,4.208832e+00,0.000000,11.427112,1.643844,0.033516
pow sum,2.388689e-17,102.666667,4187.000000,3.723869e-17,6.614798,264.916591,0.192799,0.017071
rosenbrock,2.850711e+01,1000.000000,40948.900000,1.459359e-01,0.000000,12.827964,1.922184,0.041907


In [12]:

def compare_algorithms(algo1_name: str, algo2_name: str, test_name: str, n: int = 30):
    algo1_last_val_array, algo1_iter_to_vtr_arr, algo1_nfev_arr, algo1_time_arr = read_all_data_for_test(algo1_name, test_name, n)
    algo2_last_val_array, algo2_iter_to_vtr_arr, algo2_nfev_arr, algo2_time_arr = read_all_data_for_test(algo2_name, test_name, n)

    last_val_equal_ind = algo1_last_val_array == algo2_last_val_array
    iter_to_vtr_equal_ind = algo1_iter_to_vtr_arr == algo2_iter_to_vtr_arr
    nfev_equal_ind = algo1_nfev_arr == algo2_nfev_arr
    time_equal_ind = algo1_time_arr == algo2_time_arr

    last_val_less = None
    iter_to_vtr_less = None
    nfev_less = None
    time_less = None

    if last_val_equal_ind.shape[0] == 0 or not np.all(last_val_equal_ind):
        last_val_less = stats.wilcoxon(algo1_last_val_array, algo2_last_val_array, alternative="less")

    if iter_to_vtr_equal_ind.shape[0] == 0 or not np.all(iter_to_vtr_equal_ind):
        iter_to_vtr_less = stats.wilcoxon(algo1_iter_to_vtr_arr, algo2_iter_to_vtr_arr, alternative="less")

    if nfev_equal_ind.shape[0] == 0 or not np.all(nfev_equal_ind):
        nfev_less = stats.wilcoxon(algo1_nfev_arr, algo2_nfev_arr, alternative="less")

    if time_equal_ind.shape[0] == 0 or not np.all(time_equal_ind):
        time_less = stats.wilcoxon(algo1_time_arr, algo2_time_arr, alternative="less")

    return last_val_less, iter_to_vtr_less, nfev_less, time_less


In [13]:
compare_algorithms("ttp_de_sa", "ge", "rosenbrock")


(WilcoxonResult(statistic=0.0, pvalue=9.313225746154785e-10),
 WilcoxonResult(statistic=0.0, pvalue=9.313225746154785e-10),
 WilcoxonResult(statistic=465.0, pvalue=1.0),
 WilcoxonResult(statistic=465.0, pvalue=1.0))

In [14]:
statistics_test_columns = ["last val pvalue", "iter to vtr pvalue", "nfev pvalue", "time pvalue"]

def make_stats_table(algo1_name: str, algo2_name: str, test_names: List[str] = test_names):
    statistics_test_df = pd.DataFrame(columns=statistics_test_columns)
    for test_name in test_names:
        last_val_less, iter_to_vtr_less, nfev_less, time_less = compare_algorithms(algo1_name, algo2_name, test_name)
        last_val_pvalue = last_val_less.pvalue if last_val_less is not None else None
        iter_to_vtr_pvalue = iter_to_vtr_less.pvalue if iter_to_vtr_less is not None else None
        nfev_pvalue = nfev_less.pvalue if nfev_less is not None else None
        time_pvalue = time_less.pvalue if time_less is not None else None
        statistics_test_df.loc[test_name.replace("_", " ")] = [last_val_pvalue, iter_to_vtr_pvalue, nfev_pvalue, time_pvalue]
    return statistics_test_df.fillna(1.0)

def make_stats_table_all_for_one(algo_name: Tuple[str, str], comparing_algo_names: List[Tuple[str,str]], test_names: List[str] = test_names):
    latex_table_rows = ""
    latex_row_template = """
\\thead{FUNC_NAME}
    &   \\thead{VAL_GE}
        &   \\thead{VAL_DE}
            &   \\thead{VAL_DA}
                &   \\thead{VAL_ABC}
                    &   \\thead{VAL_PSO}\\\\"""
    algo_name_only = algo_name[0]
    for test in test_names:
        latex_row = latex_row_template.replace("FUNC_NAME", test.replace("_", " "))
        for comparing_algo_name, comparing_algo_label in comparing_algo_names:
            stat_df = make_stats_table(algo_name_only, comparing_algo_name, [test])
            latex_row = latex_row.replace(f"VAL_{comparing_algo_label}", f"{stat_df.loc[test.replace('_', ' ')]['last val pvalue']:.2e}")
        latex_table_rows += latex_row
    print(latex_table_rows)
    return latex_table_rows


In [15]:
import warnings
warnings.filterwarnings("ignore")
res = make_stats_table_all_for_one(("ttp_de_sa_2", "TTP"), [("ge", "GE"), ("de", "DE"), ("da", "DA"), ("abc", "ABC"), ("pso", "PSO")])


\thead{sphere}
    &   \thead{9.31e-10}
        &   \thead{9.31e-10}
            &   \thead{9.31e-10}
                &   \thead{9.31e-10}
                    &   \thead{9.31e-10}\\
\thead{weighted sphere}
    &   \thead{9.31e-10}
        &   \thead{9.31e-10}
            &   \thead{9.31e-10}
                &   \thead{9.31e-10}
                    &   \thead{9.31e-10}\\
\thead{schwefel 1 2}
    &   \thead{9.31e-10}
        &   \thead{9.31e-10}
            &   \thead{9.31e-10}
                &   \thead{9.31e-10}
                    &   \thead{9.31e-10}\\
\thead{schwefel 2 3}
    &   \thead{9.31e-10}
        &   \thead{9.31e-10}
            &   \thead{9.31e-10}
                &   \thead{9.31e-10}
                    &   \thead{9.31e-10}\\
\thead{rotated hyper ellipsoid}
    &   \thead{9.31e-10}
        &   \thead{9.31e-10}
            &   \thead{9.31e-10}
                &   \thead{9.31e-10}
                    &   \thead{9.31e-10}\\
\thead{rosenbrock}
    &   \thead{9.31e-10}
       

In [16]:
latex_row_template = """
\\thead{FUNC_NAME}
    &   \\thead{VAL_TTP}
        &   \\thead{VAL_GE}
            &   \\thead{VAL_DE}
                &   \\thead{VAL_DA}
                    &   \\thead{VAL_ABC}
                        &   \\thead{VAL_PSO}\\\\"""

def make_latex_table_for_value(
        algo_names: List[Tuple[str, str]],
        column_name: str,
        test_names: List[str] = test_names,
        number_format: str = ".2e"
):
    latex_table_rows = ""
    val_for_each_algo_map = {}

    column_name_mean = f"{column_name} mean"
    column_name_iter = f"iter to vtr mean"
    column_name_std = f"{column_name} std"

    for algo_name, algo_label in algo_names:
        values_df = make_values_table(algo_name, test_names)
        val_for_each_algo_map[algo_label] = values_df


    for test_name in test_names:
        latex_row = latex_row_template.replace("FUNC_NAME", test_name.replace("_", " "))
        test_name_norm = test_name.replace("_", " ")
        min_val_algo = None
        min_val = np.inf

        terminated_before_vtr_map = {}

        for algo_label in val_for_each_algo_map.keys():
            mean_val = val_for_each_algo_map[algo_label].loc[test_name_norm][column_name_mean]
            if val_for_each_algo_map[algo_label].loc[test_name_norm][column_name_iter] == 1000:
                terminated_before_vtr_map[algo_label] = mean_val
            elif mean_val < min_val and val_for_each_algo_map[algo_label].loc[test_name_norm][column_name_iter] < 1000:
                min_val = mean_val
                min_val_algo = algo_label

        for algo_label in val_for_each_algo_map.keys():
            # use scientific notation for values
            mean_val = f"{val_for_each_algo_map[algo_label].loc[test_name_norm][column_name_mean]:{number_format}}"
            std_val = f"{val_for_each_algo_map[algo_label].loc[test_name_norm][column_name_std]:{number_format}}"

            if algo_label in terminated_before_vtr_map.keys():
                cell = "\\makecell{" + f"{mean_val} \\\\ ({std_val})*" + "}"
            elif algo_label == min_val_algo:
                cell = "\\textbf{\\makecell{" + f"{mean_val} \\\\ ({std_val})" + "}}"
            else:
                cell = "\\makecell{" + f"{mean_val} \\\\ ({std_val})" + "}"
            latex_row = latex_row.replace(f"VAL_{algo_label}", cell)
        latex_table_rows += latex_row
    print(latex_table_rows)
    return latex_table_rows

In [17]:
algo_names = [
    ("ttp_de_sa_2", "TTP"),
    ("ge", "GE"),
    ("de", "DE"),
    ("da", "DA"),
    ("abc", "ABC"),
    ("pso", "PSO")
]

In [18]:
text = make_latex_table_for_value(algo_names, "last val")


\thead{sphere}
    &   \thead{\textbf{\makecell{1.87e-62 \\ (5.43e-62)}}}
        &   \thead{\makecell{2.03e-17 \\ (8.06e-18)}}
            &   \thead{\makecell{5.69e-20 \\ (8.04e-20)}}
                &   \thead{\makecell{4.65e-13 \\ (6.56e-13)}}
                    &   \thead{\makecell{3.14e-11 \\ (4.13e-11)}}
                        &   \thead{\makecell{1.95e-05 \\ (1.03e-04)}}\\
\thead{weighted sphere}
    &   \thead{\textbf{\makecell{1.58e-59 \\ (4.12e-59)}}}
        &   \thead{\makecell{1.03e+01 \\ (4.21e+00)*}}
            &   \thead{\makecell{5.51e-19 \\ (5.80e-19)}}
                &   \thead{\makecell{1.86e-09 \\ (2.39e-09)}}
                    &   \thead{\makecell{3.27e-10 \\ (2.96e-10)}}
                        &   \thead{\makecell{3.24e+00 \\ (1.70e+01)*}}\\
\thead{schwefel 1 2}
    &   \thead{\textbf{\makecell{4.84e-17 \\ (1.74e-16)}}}
        &   \thead{\makecell{7.93e+02 \\ (3.76e+02)*}}
            &   \thead{\makecell{3.11e+03 \\ (7.94e+02)*}}
                &   \t

In [19]:
text = make_latex_table_for_value(algo_names, "iter to vtr", number_format=".3f")


\thead{sphere}
    &   \thead{\makecell{3.300 \\ (2.397)}}
        &   \thead{\makecell{182.333 \\ (9.217)}}
            &   \thead{\makecell{487.967 \\ (10.959)}}
                &   \thead{\textbf{\makecell{1.000 \\ (0.000)}}}
                    &   \thead{\makecell{713.033 \\ (45.066)}}
                        &   \thead{\makecell{983.433 \\ (23.410)}}\\
\thead{weighted sphere}
    &   \thead{\makecell{48.733 \\ (5.876)}}
        &   \thead{\makecell{1000.000 \\ (0.000)*}}
            &   \thead{\makecell{532.500 \\ (11.555)}}
                &   \thead{\textbf{\makecell{1.000 \\ (0.000)}}}
                    &   \thead{\makecell{794.700 \\ (33.971)}}
                        &   \thead{\makecell{1000.000 \\ (0.000)*}}\\
\thead{schwefel 1 2}
    &   \thead{\makecell{60.933 \\ (11.048)}}
        &   \thead{\makecell{1000.000 \\ (0.000)*}}
            &   \thead{\makecell{1000.000 \\ (0.000)*}}
                &   \thead{\textbf{\makecell{1.000 \\ (0.000)}}}
                    &   

In [20]:
text = make_latex_table_for_value(algo_names, "nfev", number_format=".3f")


\thead{sphere}
    &   \thead{\makecell{525.400 \\ (229.006)}}
        &   \thead{\makecell{7373.667 \\ (368.829)}}
            &   \thead{\makecell{19598.667 \\ (438.359)}}
                &   \thead{\textbf{\makecell{185.000 \\ (0.000)}}}
                    &   \thead{\makecell{28541.333 \\ (1802.649)}}
                        &   \thead{\makecell{39337.333 \\ (936.415)}}\\
\thead{weighted sphere}
    &   \thead{\makecell{11915.533 \\ (1813.344)}}
        &   \thead{\makecell{40246.433 \\ (11.427)*}}
            &   \thead{\makecell{21380.000 \\ (462.198)}}
                &   \thead{\textbf{\makecell{1206.967 \\ (71.360)}}}
                    &   \thead{\makecell{31808.000 \\ (1358.829)}}
                        &   \thead{\makecell{40000.000 \\ (0.000)*}}\\
\thead{schwefel 1 2}
    &   \thead{\makecell{14297.633 \\ (2055.951)}}
        &   \thead{\makecell{40378.267 \\ (11.051)*}}
            &   \thead{\makecell{40040.000 \\ (0.000)*}}
                &   \thead{\textbf{\makece

In [21]:
text = make_latex_table_for_value(algo_names, "time", number_format=".3f")


\thead{sphere}
    &   \thead{\makecell{0.010 \\ (0.004)}}
        &   \thead{\makecell{0.323 \\ (0.026)}}
            &   \thead{\makecell{0.486 \\ (0.025)}}
                &   \thead{\textbf{\makecell{0.004 \\ (0.000)}}}
                    &   \thead{\makecell{0.623 \\ (0.041)}}
                        &   \thead{\makecell{2.013 \\ (0.062)}}\\
\thead{weighted sphere}
    &   \thead{\makecell{0.136 \\ (0.017)}}
        &   \thead{\makecell{1.644 \\ (0.034)*}}
            &   \thead{\makecell{0.391 \\ (0.009)}}
                &   \thead{\textbf{\makecell{0.012 \\ (0.001)}}}
                    &   \thead{\makecell{0.389 \\ (0.020)}}
                        &   \thead{\makecell{1.807 \\ (0.018)*}}\\
\thead{schwefel 1 2}
    &   \thead{\textbf{\makecell{0.770 \\ (0.110)}}}
        &   \thead{\makecell{13.678 \\ (0.159)*}}
            &   \thead{\makecell{11.750 \\ (0.050)*}}
                &   \thead{\makecell{1.277 \\ (0.139)}}
                    &   \thead{\makecell{12.470 \\ (0.